In [1]:
import cv2
import mediapipe

import time
import serial
import pyfirmata
#ser.close()

from enum import Enum

class States(Enum): #enumarition of states 
    STOP=1
    GOFORWARD=2
    GOREVERSE=3
    TURNRIGHT=4
    TURNLEFT=5

prev_value=States.STOP

class transition():      #transition class to change states 
    def __init__(self):
        pass
        
    def go_to_state(self,state_name):
        self.state_name=state_name
        
    def check_state(self):
        return self.state_name
    
    def prev_state(self):
        pass
      

command=transition() #creating abject for transition

def show_changing_state(prev_state,next_state):
    cv2.putText(img,str(prev_state),(30,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(50,168,82),2)
    cv2.rectangle(img,(10,70),(200,120),(50,168,82),cv2.FILLED)
    cv2.putText(img,str(next_state),(30,95),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
    
    
    

ArduinoSerial = serial.Serial('COM27', 9600, timeout= 0.01)                     #Creating our Arduino serial object
ctime=0
ptime=0

cap=cv2.VideoCapture(0)

medhands=mediapipe.solutions.hands
hands=medhands.Hands(max_num_hands=1,min_detection_confidence=0.8)
draw=mediapipe.solutions.drawing_utils

command.go_to_state(States.STOP) #Starting the FSM in the STOP state
prev_value=command.check_state()
transit_value = command.check_state()
while True:
    success, img=cap.read()
    img = cv2.flip(img,1)
    imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
    res = hands.process(imgrgb)
    
    
    
    
    lmlist=[]
    tipids=[4,8,12,16,20] #list of all landmarks of the tips of fingers
    
    #cv2.rectangle(img,(20,350),(90,440),(0,255,204),cv2.FILLED)
    #cv2.rectangle(img,(20,350),(90,440),(0,0,0),5)
    
    if res.multi_hand_landmarks:
        for handlms in res.multi_hand_landmarks:
            for id,lm in enumerate(handlms.landmark):
                
                h,w,c= img.shape
                cx,cy=int(lm.x * w) , int(lm.y * h)
                lmlist.append([id,cx,cy])
                if len(lmlist) != 0 and len(lmlist)==21:
                    fingerlist=[]
                    
                    #thumb and dealing with flipping of hands
                    if lmlist[12][1] > lmlist[20][1]:
                        if lmlist[tipids[0]][1] > lmlist[tipids[0]-1][1]:
                            fingerlist.append(1)
                        else:
                            fingerlist.append(0)
                    else:
                        if lmlist[tipids[0]][1] < lmlist[tipids[0]-1][1]:
                            fingerlist.append(1)
                        else:
                            fingerlist.append(0)
                    
                    #others
                    for id in range (1,5):
                        if lmlist[tipids[id]][2] < lmlist[tipids[id]-2][2]:
                            fingerlist.append(1)
                        else:
                            fingerlist.append(0)
                    
                    
                    if len(fingerlist)!=0:
                        fingercount=fingerlist.count(1)
                    
                    
                    #cv2.putText(img,str(fingercount),(25,430),cv2.FONT_HERSHEY_PLAIN,6,(0,0,0),5)
                    
                    #board = pyfirmata.Arduino('COM43')
                    #if fingercount == 5:
                        #board.digital[12].write(1)
                    #else:
                        #board.digital[12].write(0)
                    
                    
                    #communitcating with arduino using serial
                    #with serial.Serial('COM43', 9600, timeout=1) as ser:
                        #time.sleep(1.0)
                    
                    if prev_value != command.check_state():
                    
                        if transit_value != command.check_state():
                            
                            prev_value = transit_value
                            transit_value = command.check_state()
                        
                    
                    if fingercount == 5:
                        command.go_to_state(States.GOFORWARD);
                        ArduinoSerial.write(b'H')   # send the pyte string 'H'
                        cv2.putText(img,"Forward",(390,430),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),2)
                        
                        #if (ArduinoSerial.inWaiting()):
                        data = ArduinoSerial.readline()
                        # print(data.decode("utf-8"))
                        # print(command.check_state())
                        if command.check_state() == States.GOFORWARD:
                            #print("robot MOVING FORWARD")
                            show_changing_state(prev_value,command.check_state())
                        
                    elif fingercount == 1:
                        command.go_to_state(States.TURNLEFT);
                        ArduinoSerial.write(b'J')   # send the pyte string 'H'
                        cv2.putText(img,"Left",(390,430),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),2)
                        
                        #if (ArduinoSerial.inWaiting()):
                        #data = ArduinoSerial.readline()
                        #print(data.decode("utf-8"))
                        print(command.check_state())
                        if command.check_state() == States.TURNLEFT:
                            print("robot TURNING LEFT")
                            show_changing_state(prev_value,command.check_state())
                        
                        
                    elif fingercount == 2:
                        command.go_to_state(States.TURNRIGHT)
                        ArduinoSerial.write(b'R')   # send the pyte string 'H'
                        cv2.putText(img,"Right",(390,430),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),2)
                        
                        #if (ArduinoSerial.inWaiting()):
                        #data = ArduinoSerial.readline()
                        #print(data.decode("utf-8"))
                        print(command.check_state())
                        if command.check_state() == States.TURNRIGHT:
                            print("robot TURNING RIGHT")
                            show_changing_state(prev_value,command.check_state())
                            
                    elif fingercount == 4:
                        command.go_to_state(States.GOREVERSE)
                        ArduinoSerial.write(b'O')   # send the pyte string 'H'
                        cv2.putText(img,"Reverse",(390,430),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),2)
                        
                        #if (ArduinoSerial.inWaiting()):
                        #data = ArduinoSerial.readline()
                        #print(data.decode("utf-8"))
                        print(command.check_state())
                        if command.check_state() == States.GOREVERSE:
                            print("robot MOVING REVERSE")
                            show_changing_state(prev_value,command.check_state())
                        
                    elif fingercount == 0:
                        command.go_to_state(States.STOP)
                        ArduinoSerial.write(b'L')   # send the pyte string 'H'
                        cv2.putText(img,"Stop",(390,430),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),2)
                        
                        #if (ArduinoSerial.inWaiting()):
                        data = ArduinoSerial.readline()
                        #print(data.decode("utf-8"))
                        print(command.check_state())
                        if command.check_state() == States.STOP:
                            print("robot STOPPING")
                            show_changing_state(prev_value,command.check_state())
                    #if (ArduinoSerial.inWaiting()):
                    #print(ArduinoSerial.read())
                       
                    
                        
                        
                #change color of points and lines
                draw.draw_landmarks(img,handlms,medhands.HAND_CONNECTIONS,draw.DrawingSpec(color=(0,255,204),thickness=2,circle_radius=2),draw.DrawingSpec(color=(0,0,0),thickness=2,circle_radius=3))
    
    #fps counter
    ctime = time.time()
    fps=1/(ctime-ptime)
    ptime=ctime
    
    #fps display
    cv2.putText(img,f'FPS:{str(int(fps))}',(0,12),cv2.FONT_HERSHEY_PLAIN,1,(0,255,0),1)
    img = cv2.resize(img, None, fx=1.25, fy=1.25)
    cv2.imshow("hand gestures",img)
    
    #press q to quit
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

States.TURNRIGHT
robot TURNING RIGHT
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.TURNLEFT
robot TURNING LEFT
States.TURNLEFT
robot TURNING LEFT
St

States.GOREVERSE
robot MOVING REVERSE
States.GOREVERSE
robot MOVING REVERSE
States.GOREVERSE
robot MOVING REVERSE
States.GOREVERSE
robot MOVING REVERSE
States.GOREVERSE
robot MOVING REVERSE
States.TURNLEFT
robot TURNING LEFT
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.TURNLEFT
rob

States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING
States.STOP
robot STOPPING


KeyboardInterrupt: 